In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# ==========================================
#  MFCC vs. HuBERT Feature Comparison
# ==========================================

# Purpose: Compare the performance of traditional MFCC features and HuBERT embeddings
# on the same Native Language Identification (Accent Classification) task.

# ------------------------------------------
# Import required libraries
# ------------------------------------------
import numpy as np
import os
import librosa
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# ------------------------------------------
# 📂 Dataset and Paths
# ------------------------------------------
DATA_PATH = "/content/drive/MyDrive/Accent1/Accent-Aware-Cuisine-Recommendation/IndicAccentDB"
SAVE_PATH = "/content/drive/MyDrive/Accent1/Accent-Aware-Cuisine-Recommendation"

states = sorted(os.listdir(DATA_PATH))
print("🎧 Languages/States detected:", states)

# ------------------------------------------
# STEP 1: Load existing HuBERT features
# ------------------------------------------
hubert_features = np.load(f"{SAVE_PATH}/hubert_features.npy")
hubert_labels = np.load(f"{SAVE_PATH}/hubert_labels.npy")

print("✅ Loaded HuBERT features:", hubert_features.shape)
print("✅ Loaded HuBERT labels:", hubert_labels.shape)

# ------------------------------------------
#STEP 2: Extract MFCC features (if not already saved)
# ------------------------------------------
mfcc_features, mfcc_labels = [], []

for label, state in enumerate(states):
    folder = os.path.join(DATA_PATH, state)
    wav_files = [f for f in os.listdir(folder) if f.endswith(".wav")]

    for file in tqdm(wav_files[:50], desc=f"MFCC {state}"):  # limit to 50 per state for speed
        try:
            wav_path = os.path.join(folder, file)
            wav, sr = librosa.load(wav_path, sr=16000)
            mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=40)
            mfcc_mean = np.mean(mfcc, axis=1)
            mfcc_features.append(mfcc_mean)
            mfcc_labels.append(label)
        except Exception as e:
            print(f"⚠️ Error with {file}: {e}")

mfcc_features = np.array(mfcc_features)
mfcc_labels = np.array(mfcc_labels)

# Save for future use
np.save(f"{SAVE_PATH}/mfcc_features.npy", mfcc_features)
np.save(f"{SAVE_PATH}/mfcc_labels.npy", mfcc_labels)

print("✅ MFCC feature extraction complete!")
print("MFCC Features shape:", mfcc_features.shape)
print("MFCC Labels shape:", mfcc_labels.shape)

# ------------------------------------------
#  STEP 3: Train models on both feature sets
# ------------------------------------------

def train_and_evaluate(X, y, feature_type):
    """Train RandomForest and return accuracy."""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred) * 100
    print(f"\n {feature_type} Accuracy: {acc:.2f}%")
    print(f"{feature_type} Classification Report:\n", classification_report(y_test, y_pred))

    return acc


# Train and evaluate MFCC model
mfcc_acc = train_and_evaluate(mfcc_features, mfcc_labels, "MFCC")

# Train and evaluate HuBERT model
hubert_acc = train_and_evaluate(hubert_features, hubert_labels, "HuBERT")

# ------------------------------------------
# STEP 4: Compare and summarize results
# ------------------------------------------
comparison_df = pd.DataFrame({
    "Feature Type": ["MFCC", "HuBERT"],
    "Accuracy (%)": [mfcc_acc, hubert_acc]
})

print("\n Model Comparison Summary:")
print(comparison_df)

# Optional: Save results to CSV for report
comparison_df.to_csv(f"{SAVE_PATH}/feature_comparison_results.csv", index=False)
print("✅ Comparison results saved!")


🎧 Languages/States detected: ['.ipynb_checkpoints', 'andhra_pradesh', 'gujarat', 'jharkhand', 'karnataka', 'kerala', 'tamil']
✅ Loaded HuBERT features: (8116, 768)
✅ Loaded HuBERT labels: (8116,)


MFCC .ipynb_checkpoints: 0it [00:00, ?it/s]
MFCC tamil: 100%|██████████| 50/50 [00:10<00:00,  4.85it/s]


✅ MFCC feature extraction complete!
MFCC Features shape: (300, 40)
MFCC Labels shape: (300,)

 MFCC Accuracy: 95.00%
MFCC Classification Report:
               precision    recall  f1-score   support

           1       1.00      0.70      0.82        10
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        10
           5       0.91      1.00      0.95        10
           6       0.91      1.00      0.95        10

    accuracy                           0.95        60
   macro avg       0.95      0.95      0.95        60
weighted avg       0.95      0.95      0.95        60


 HuBERT Accuracy: 97.84%
HuBERT Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97       359
           1       1.00      0.92      0.96        60
           2       1.00      0.95      0.98       166
           3       0.98      0.99    